# Weekly Project 5
## Global Registration implementation.
## Task 1
Today your project is to implement a global registration algorithm.

It should be able to roughly align two pointclouds.
1. Implement global registration. You are allowed to use the FPHF features and have to implement the RANSAC algorithm

2. Can you fit **r1.pcd** and **r2.pcd**?
3. Can you fit **car1.ply** and **car2.ply**?
These are in the *global_registration* folder



## Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in *car_challange* folder.

You can use the exercises from monday as a starting point.

In [1]:
# from ex1 

import open3d as o3d
import numpy as np
import copy

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
    
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(voxel_size, source, target):

    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registrations(source, target)

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result


# Start tasks

## Task 1.1

Implementation of RANSAC


In [2]:
r1 = o3d.io.read_point_cloud("global_registration/r1.pcd")
r2 = o3d.io.read_point_cloud("global_registration/r2.pcd")

# Used for downsampling.
voxel_size = 0.05

voxel_size = 0.05  # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, r1, r2)



:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


In [10]:
print(source_fpfh)

Feature class with dimension = 33 and num = 4760
Access its data via data member.


In [82]:
# FIND THE PAIRS

s = source_fpfh.data
t = target_fpfh.data

# sourcepoints = source_fpfh.num()
sourcepoints =  100

dist_t = np.zeros([target_fpfh.num(),1])
index = []

for u in range(sourcepoints):
    ps = s[:,u]

    for i in range(target_fpfh.num()):
        pt = t[:,i]
        dist_t[i] = np.linalg.norm(ps-pt)
        
    if u >= target_fpfh.num():
        break
    index.append(np.argmin(dist_t))
    
    
        

pairs_target_points = np.asarray(target_down.points)[index,:]

print(index)






[1348, 3154, 1442, 1348, 2987, 2346, 2947, 3165, 3259, 2969, 2969, 3036, 2520, 1878, 1906, 122, 2136, 2500, 2129, 2927, 2927, 2930, 1592, 1798, 1967, 1311, 2035, 2336, 2634, 2677, 236, 236, 309, 308, 584, 1003, 1856, 2413, 127, 2677, 2953, 1455, 2573, 155, 3180, 2570, 1526, 220, 1233, 2634, 610, 209, 209, 271, 129, 157, 2964, 2718, 2144, 3360, 495, 1553, 1592, 2688, 3245, 194, 2685, 195, 2250, 2724, 143, 2214, 185, 1798, 1591, 780, 931, 3276, 1988, 237, 3094, 545, 1253, 185, 185, 185, 100, 142, 1069, 1520, 1072, 2554, 3154, 2900, 2960, 2847, 3083, 1139, 2214, 2074]


In [80]:
index = int(index)
da = int(index[1,0])
print((type(da)))

TypeError: only size-1 arrays can be converted to Python scalars

1. find pairs in fpfh (source <-> target)
2. take N number of pairs and run Kabsch -> rot, trans = tranformation
3. applying transformation
4. calc distance of pairs and sum = error
5. find best model

 
### PSEUDOCODE
 
 ```
 RANSAC(k=maxinter, n= numberofrandompoints, source_fpfh, target)
 
 for ALL source
     onesource
     for all target
         onetarget
         dist = norm(1sou-1tar)
 
 vector_sorted_source
 vector _ target
 
 while iter < k:
 
 
     find n pairs
 
     points = rand n
     
     rotmat,transvec = kabsch(points)
     
     target_transformed = rotmat*target + transvec
     
     error = norm2( source, target)
     
     if error < bestErr then
            bestFit := r,t
            bestErr := thisErr
     end if
     
     return bestFit
 
 ```
        





In [27]:
target_fpfh.num()

3440

In [43]:
n = np.rand(0,source_fpfh.num())

AttributeError: module 'numpy' has no attribute 'rand'

In [58]:
print(np.asarray(source_down.points).shape)
print(np.asarray(target_down.points).shape)


(4760, 3)
(3440, 3)


In [54]:
target_fpfh.shape
source_down.shape

AttributeError: 'open3d.cpu.pybind.pipelines.registration.Feature' object has no attribute 'shape'

In [61]:
4760*3440


16374400